# StableLM SageMaker Finetuning

This is a sample code to finetune and deploy StableLM with LoRA on SageMaker.

In [ ]:
!pip install -U "sagemaker>=2.143.0"

In [ ]:
import sagemaker, boto3, json
from sagemaker import get_execution_role
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.huggingface import HuggingFace

role = get_execution_role()
region = boto3.Session().region_name
sess = sagemaker.Session()
bucket = sess.default_bucket()

sagemaker.__version__

## Upload Data

We will use Databricks-dolly-15k as sample dataset to finetune the model. (License: [Creative Commons Attribution-ShareAlike 3.0 Unported License](https://creativecommons.org/licenses/by-sa/3.0/legalcode))

You may also choose to use custom dataset.

In [ ]:
!curl -L https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl --create-dirs -o data/databricks-dolly-15k.jsonl

In [ ]:
!head -n 2 data/databricks-dolly-15k.jsonl

In [ ]:
# Convet .jsonl to .json
import pandas as pd
df = pd.read_json('data/databricks-dolly-15k.jsonl', orient='records', lines=True)
df = df.rename(columns={"context": "input", "response": "output"})
df.to_json("data/databricks-dolly-15k.json", orient='records')

In [ ]:
input_train = sess.upload_data(
    path="./data/databricks-dolly-15k.json",
    key_prefix="Dolly"
)
input_train

## Fine-tuning


In [ ]:
hyperparameters={
    'base_model':'stabilityai/stablelm-base-alpha-3b',
    'load_in_8bit': False,
    'data_path': '/opt/ml/input/data/train/databricks-dolly-15k.json',
    'num_epochs': 3, # default 3
    'cutoff_len': 512,
    'group_by_length': True,
    'output_dir': '/opt/ml/model',
    'lora_target_modules': '[query_key_value]',
    'lora_r': 16,
    'batch_size': 32,
    'micro_batch_size': 4,
    'prompt_template_name': 'stable_lm',
    'pad_token_id': 1,
}

In [ ]:
huggingface_estimator = HuggingFace(
    base_job_name="StableLM",
    role=role,
    entry_point='finetune.py',
    source_dir='./scripts/code',
    instance_type='ml.g5.2xlarge',
    instance_count=1,
    volume_size=200,
    transformers_version='4.26',
    pytorch_version='1.13',
    py_version='py39',
    use_spot_instances=True,
    max_wait=86400,
    hyperparameters=hyperparameters,
)
huggingface_estimator.fit({'train': input_train})

## Download and Extract Model

In [ ]:
import boto3
import sagemaker

def get_latest_training_job_artifact(base_job_name):
    sagemaker_client = boto3.client('sagemaker')
    response = sagemaker_client.list_training_jobs(NameContains=base_job_name, SortBy='CreationTime', SortOrder='Descending')
    training_job_arn = response['TrainingJobSummaries'][0]['TrainingJobArn']
    training_job_description = sagemaker_client.describe_training_job(TrainingJobName=training_job_arn.split('/')[-1])
    return training_job_description['ModelArtifacts']['S3ModelArtifacts']

try:
    model_data = huggingface_estimator.model_data
except:
    # Retrieve artifact url when kernel is restarted
    model_data = get_latest_training_job_artifact('StableLM')
    
!aws s3 cp {model_data} stable_lm.tar.gz

In [ ]:
!rm -rf scripts/model && mkdir scripts/model
!tar -xvf stable_lm.tar.gz -C scripts/model --no-same-owner --wildcards adapter_*

## Package and Upload Model

In [ ]:
%cd scripts
!tar -czvf ../package.tar.gz *
%cd -

In [ ]:
model_path = sess.upload_data('package.tar.gz', bucket=bucket, key_prefix=f"StableLM")
model_path

## Deploy Model

In [ ]:
from sagemaker.async_inference import AsyncInferenceConfig
from sagemaker.serializers import JSONSerializer

endpoint_name = "StableLM"

huggingface_model = PyTorchModel(
    model_data=model_path,
    framework_version="1.13",
    py_version='py39',
    role=role,
    name=endpoint_name,
    env={
        "model_params": json.dumps({
            "base_model": "stabilityai/stablelm-base-alpha-3b",
            "lora_weights": "model", # path relative to model package
            "peft": True,
            "load_8bit": True,
            "prompt_template": "stable_lm",
        }),
        "SAGEMAKER_MODEL_SERVER_TIMEOUT": "600"
    }
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type='ml.g5.2xlarge',
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
    async_inference_config=AsyncInferenceConfig()
)

## Run Inference

In [ ]:
from sagemaker.predictor import Predictor
from sagemaker.predictor_async import AsyncPredictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import NumpyDeserializer

predictor_client = AsyncPredictor(
    predictor=Predictor(
        endpoint_name=endpoint_name,
        sagemaker_session=sess,
        serializer=JSONSerializer(),
        deserializer=NumpyDeserializer()
    ),
    name=endpoint_name
)
data = {
    "instruction": """# StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
- StableLM will refuse to participate in anything that could harm a human.
""",
    "input": """What's your mood today?
""",
    "max_new_tokens": 64,
    "temperature": 0.7,
    "do_sample": True,
    "pad_token_id": 1,
    "bos_token_id": 0,
    "eos_token_id": 0,
    "stop_ids": [50278, 50279, 50277, 1, 0],
}
response = predictor_client.predict(
    data=data
)
print(response)

## Delete Endpoint

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()